In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as go2
import nbformat
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import date
from dateutil.relativedelta import relativedelta

## Get All Stocks in the S&P 500

In [ ]:

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url)
sp500_table = tables[0]

# Get Tickers
sp500_table = sp500_table['Symbol'].tolist()

print(sp500_table)


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'XYZ', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DDOG', '

### Create Algorithm Logic
- Mean Reversion + Momentum:
    - WHEN Low < Bollinger Band Lower AND (RSI < 30 OR MFI < 20) THEN "BUY"
    - WHEN High > Bollinger Band Upper AND (RSI > 70 OR MFI > 80) THEN "SELL"
    
- Bollinger Bands

$$
EMA_t = \alpha \times (Close_t - EMA_{t-1}) + EMA_{t-1}
$$

$$
Upper Band=Middle Band EMA +(nxStandard Deviation)
$$

- RSI
$$
RSI = 100 - (\dfrac{100}{1+RS})
$$

$$
WhereRS = \dfrac{Average(gain)}{Average(Loss)}
$$

- Money Flow Index (MFI)
- An oscillator that ranges from 1 to 100. It is used to show the money flow (an approx. of the follor value of a day's trading) over several days
$$
typical price = \dfrac{high+low+close}{3}
$$

$$
money ratio = \dfrac{positive money flow}{negative money flow}
$$

$$
MFI = 100 - \dfrac{100}{1 + money ratio}
$$

In [ ]:
# Get today's Date
end = date.today()

# Subtract 5 years. Back test will be 5yrs worth of data
start = end - relativedelta(years = 5)

def bollinger_bands(df,window=20):
    df[f'SMA{window}'] = df['Close'].rolling(window=window).mean()
    df[f'STD{window}'] = df['Close'].rolling(window=window).std()

    df[f'Upper Band {window}'] = df[f'SMA{window}'] + (df[f'STD{window}']*2)
    df[f'Lower Band {window}'] = df[f'SMA{window}'] - (df[f'STD{window}']*2)

    return df

def create_rsi(df, window=14):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0,0)) #Separate gains from losses
    loss = (-delta.where(delta < 0,0))

    # Calc avg gain and loss
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    rs = avg_gain/avg_loss
    rsi_value = 100-(100/(1+rs))

    df['rsi'] = rsi_value
    return df

def money_flow_index(df,window=14):
    typical_price = (df['High'] + df['Low'] + df['Close'])/3

    # Calc Raw Money Flow
    raw_money_flow = typical_price*df['Volume']

    # Calc (+) and (-) money flow
    positive_flow = raw_money_flow.where(typical_price > typical_price.shift(1),0)
    negative_flow = raw_money_flow.where(typical_price < typical_price.shift(1),0)

    # Calc Money Flow Ratio
    window = 14
    positive_mf_sum = positive_flow.rolling(window=window).sum()
    negative_mf_sum = negative_flow.rolling(window=window).sum()
    money_flow_ratio = positive_mf_sum/negative_mf_sum

    # Calc MFI
    mfi = 100-(100/(1+money_flow_ratio))

    # Add mfi to df
    df['mfi'] = mfi

    return df

# Custom function with trading alogrithm logic
def algorithm(ticker, start=start, end=end, interval='1d'):
    df = yf.download(ticker, start=start, end=end, interval=interval, auto_adjust=True)
    df.columns=[col[0] for col in df.columns]

    # Calc EMA and BB
    df = bollinger_bands(df=df, window=12)
    df = bollinger_bands(df=df, window=20)
    df = bollinger_bands(df=df, window=50)

    # Calc RSI for momentum
    df = create_rsi(df=df, window=14)

    # Calc MFI for momentum
    df = money_flow_index(df=df,window=14)

    # Create Logic for buy and sell signals
    # Initialize values
    low = df['Low']
    high = df['High']
    sma12 = df['SMA12']
    sma20 = df['SMA20']
    sma50 = df['SMA50']
    bollingerUpper20 = df['Upper Band 20']
    bollingerLower20 = df['Lower Band 20']
    bollingerUpper50 = df['Upper Band 50']
    bollingerLower50 = df['Lower Band 50']
    rsi = df['rsi']
    mfi = df['mfi']

    # Initialize buy_signal, sell_signal, action list
    buy_signal = []
    sell_signal = []
    action = []

    for i in range(len(df)):
        low_val = low.iloc[i]
        high_val = high.iloc[i]
        sma12_val = sma12.iloc[i]
        sma20_val = sma20.iloc[i]
        sma50_val = sma50.iloc[i]
        bollingerlower_val20 = bollingerLower20.iloc[i]
        bollingerupper_val20 = bollingerUpper20.iloc[i]
        bollingerlower_val50 = bollingerLower50.iloc[i]
        bollingerupper_val50 = bollingerUpper50.iloc[i]
        rsi_val = rsi.iloc[i]
        mfi_val = mfi.iloc[i]
        #'''
        # Buy Logic
        buy_condition = (low_val < bollingerlower_val20) and (rsi_val < 30 and mfi_val < 25)

        # Sell Logic
        sell_condition = (high_val > bollingerlower_val20) and (rsi_val > 70 and mfi_val > 75)

        # Scatter plot values for buy and sell signals
        if buy_condition: # if buy condition is true
            buy_signal.append(low_val * 0.95)
            sell_signal.append(np.nan)
            action.append("Buy")

        elif sell_condition:
            sell_signal.append(high_val * 1.05)
            buy_signal.append(np.nan)
            action.append("Sell")

        else:
            buy_signal.append(np.nan)
            sell_signal.append(np.nan)
            action.append("Hold")

    # Add new columns to df
    df['buy_signal'] = buy_signal
    df['sell_signal'] = sell_signal
    df['action'] = action

    # Back testing Logic - compare to buy and hold method
    initial_cash = 10000
    cash = initial_cash
    position = 0 #initally holding no stock
    portfolio_value = [] # Track portfolio over time

    # Buy and Hold method
    print()
    share_cost = df['Open'].iloc[0]
    num_shares = initial_cash/share_cost
    df['Buy_hold_Value'] = df['Close']*num_shares

    # Iterate over each row in df to simulate portfolio updates
    for i in range(0,len(df)):
        action = df['action'].iloc[i]
        price = df['Close'].iloc[i]

        # If action is Buy
        if action == 'Buy' and cash > 0: #Check if there is cash available
            position = cash/price #Buy as much as possible
            cash = 0

        elif action == 'Sell' and position > 0: #Check if there is stock to sell
            cash = position*price #our new cash position after selling all stock
            position = 0

        portfolio_value.append(cash+position*price)

    #add new column to df
    portfolio_value[len(df)-1] = portfolio_value[len(df)-2]
    df['Portfolio_value'] = portfolio_value
    return df

In [ ]:
df = algorithm(ticker='NVDA', start=start, end=end, interval='1d')
df

[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume,SMA12,STD12,Upper Band 12,Lower Band 12,SMA20,...,STD50,Upper Band 50,Lower Band 50,rsi,mfi,buy_signal,sell_signal,action,Buy_hold_Value,Portfolio_value
Date,,,,,,,,,,,,,,,,,,,,,
2020-08-06,11.298260,11.320686,11.142025,11.312463,244316000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hold,9987.444832,10000.000000
2020-08-07,11.162705,11.466953,11.001238,11.275334,342516000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hold,9867.617419,10000.000000
2020-08-10,11.128320,11.370770,10.820833,11.297014,427796000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hold,9837.221137,10000.000000
2020-08-11,10.814356,11.100164,10.760283,11.037370,354512000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hold,9559.683082,10000.000000
2020-08-12,11.402668,11.429330,10.922251,10.954645,464412000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hold,10079.739734,10000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-30,179.270004,179.889999,176.039993,176.509995,174312200,172.953334,3.189980,179.333295,166.573373,168.312001,...,14.543975,181.963697,123.787797,73.874718,66.238534,NaN,NaN,Hold,158471.244211,23460.161703
2025-07-31,177.869995,183.300003,175.929993,182.899994,221685400,173.550833,3.394483,180.339800,166.761867,169.343001,...,14.745410,183.212756,124.231114,70.015438,66.904927,NaN,NaN,Hold,157233.662979,23460.161703
2025-08-01,173.720001,176.539993,170.889999,174.089996,204529000,173.746667,3.324291,180.395250,167.098084,170.062001,...,14.741592,183.992107,125.025737,63.534344,64.721773,NaN,NaN,Hold,153565.147999,23460.161703


In [ ]:
# Count Frequency of buy, sell, and holds in action column
action_count = df['action'].value_counts()
print(action_count)

action
Hold    1138
Sell     107
Buy       10
Name: count, dtype: int64


## Visualize Algorithm

In [ ]:
# Combine all subplots into one figure
fig = make_subplots(
    rows =3, cols = 1,
    shared_xaxes = True,
    row_heights = [0.6, 0.2, 0.2],
    vertical_spacing = 0.02,
    subplot_titles = ("Candlestick with Buy and Sell Signals", "RSI (14)", "MFI (14)")
)

# Candlestick
fig.add_trace(go.Candlestick(
    x = df.index,
    open = df['Open'],
    high = df['High'],
    low = df['Low'],
    close = df['Close'],
    name = 'Candlestick'
))

# SMA 20
fig.add_trace(go.Scatter(
    x = df.index,
    y = df['SMA20'],
    mode = 'lines',
    name = 'SMA 20',
    line = dict(color = 'blue', width = 1, dash = 'solid')
))

# SMA 50
fig.add_trace(go.Scatter(
    x = df.index,
    y = df['SMA50'],
    mode = 'lines',
    name = 'SMA 50',
    line = dict(color = 'red', width = 1, dash = 'solid')
))

# Upper Bollinger Band
fig.add_trace(go.Scatter(
    x = df.index,
    y = df['Upper Band 20'],
    mode = 'lines',
    name = 'Upper Band 20',
    line = dict(color = 'blue', width = 1, dash = 'dash')
))

# Lower Bollinger Band
fig.add_trace(go.Scatter(
    x = df.index,
    y = df['Lower Band 20'],
    mode = 'lines',
    name = 'Lower Band 20',
    line = dict(color = 'blue', width = 1, dash = 'dash')
))

# Upper Bollinger Band
fig.add_trace(go.Scatter(
    x = df.index,
    y = df['Upper Band 50'],
    mode = 'lines',
    name = 'Upper Band 50',
    line = dict(color = 'red', width = 1, dash = 'dash')
))

# Lower Bollinger Band
fig.add_trace(go.Scatter(
    x = df.index,
    y = df['Lower Band 50'],
    mode = 'lines',
    name = 'Lower Band 50',
    line = dict(color = 'red', width = 1, dash = 'dash')
))

# Buy Signals
fig.add_trace(go.Scatter(
    x = df.index,
    y = df['buy_signal'],
    mode = 'markers',
    name = 'Buy Signal',
    marker = dict(color = 'green', symbol = 'triangle-up', size = 10)
))

# Sell Signals
fig.add_trace(go.Scatter(
    x = df.index,
    y = df['sell_signal'],
    mode = 'markers',
    name = 'Sell Signal',
    marker = dict(color = 'red', symbol = 'triangle-down', size = 10)
))

# Add RSI to 2nd Row
fig.add_trace(go.Scatter(
    x = df.index,
    y = df['rsi'],
    mode = 'lines',
    name = 'RSI',
    line = dict(color = 'purple', width = 2)
), row = 2, col = 1)

fig.add_hline(y = 70, line = dict(color = 'red', dash = 'dash'), row = 2, col = 1)
fig.add_hline(y = 30, line = dict(color = 'green', dash = 'dash'), row = 2, col = 1)

# Add MFI to 3rd Row
fig.add_trace(go.Scatter(
    x = df.index,
    y = df['mfi'],
    mode = 'lines',
    name = 'MFI',
    line = dict(color = 'brown', width = 2)
), row = 3, col = 1)

fig.add_hline(y = 80, line = dict(color = 'red', dash = 'dash'), row = 3, col = 1)
fig.add_hline(y = 20, line = dict(color = 'green', dash = 'dash'), row = 3, col = 1)

# Update Layout
fig.update_layout(
    xaxis_rangeslider_visible = False,
    template = 'plotly'
)

fig.write_html("Candlestick_Chart.html")

### Stock Screener
- Filter on stocks where the alogrithm outperformed and analysts positivly rate the stock

In [ ]:
result_dict = {}
for ticker in sp500_table:
    try: #if we run into an error for one stock, the entire script will not halt
        # Replace '.' with '-'
        ticker = ticker.replace('.', '-')
        print(f'>> Now running: {ticker}')
        df = algorithm(ticker = ticker, start=start, end = end, interval = '1d')

        # Get Wall Street analyst recommendations
        # Fetch stock data
        stock = yf.Ticker(ticker)

        # Get analyst Recs
        analyst_ratings = stock.recommendations
        analyst_ratings = analyst_ratings[(analyst_ratings['period'] == '0m')]

        count_buy = analyst_ratings['strongBuy'] + analyst_ratings['buy']
        count_buy = count_buy[0]
        count_sell = analyst_ratings['strongSell'] + analyst_ratings['sell']
        count_sell = count_sell[0]

        is_more_buys = count_buy > count_sell

        analyst_ratings['IsMoreBuys'] = ['Y' if a > b else 'N' for a, b in zip(analyst_ratings['strongBuy'] + analyst_ratings['buy'], analyst_ratings['strongSell'] + analyst_ratings['sell'])]

        result_dict[f'{ticker}'] = (df['Buy_hold_Value'].iloc[-1], df['Portfolio_value'].iloc[-1], count_buy, count_sell, analyst_ratings['IsMoreBuys'].iloc[-1])

    except Exception as e:
        print(f'>> Error: {e}') # print the error

result_dict

>> Now running: MMM


[*********************100%***********************]  1 of 1 completed


/tmp/ipython-input-1545256584.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



>> Now running: AOS


[*********************100%***********************]  1 of 1 completed


...

/tmp/ipython-input-1545256584.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



>> Now running: ZTS


[*********************100%***********************]  1 of 1 completed


/tmp/ipython-input-1545256584.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



{'MMM': (np.float64(13963.193236265499),
  np.float64(12838.794255899065),
  np.int64(11),
  np.int64(3),
  'Y'),
 'AOS': (np.float64(15993.524886874671),
  np.float64(10593.853633868079),
  np.int64(4),
  np.int64(1),
  'Y'),
 'ABT': (np.float64(14074.521336310494),
  np.float64(10346.431932914891),
  np.int64(20),
  np.int64(1),
  'Y'),
 'ABBV': (np.float64(26020.38869649255),
  np.float64(15718.863981101546),
  np.int64(16),
  np.int64(0),
  'Y'),
 'ACN': (np.float64(11704.172456140488),
  np.float64(6153.857184474861),
  np.int64(16),
  np.int64(2),
  'Y'),
 'ADBE': (np.float64(7505.040116398848),
  np.float64(15841.258829787583),
  np.int64(27),
  np.int64(2),
  'Y'),
 'AMD': (np.float64(20389.519465933445),
  np.float64(9706.596080342284),
  np.int64(36),
  np.int64(0),
  'Y'),
 'AES': (np.float64(9917.795738870753),
  np.float64(8991.605840333572),
  np.int64(8),
  np.int64(2),
  'Y'),
 'AFL': (np.float64(31593.624057856312),
  np.float64(19511.189025719625),
  np.int64(3),
  np

### Convert Back Test results into a dataframe from dictionary

In [ ]:
df_results = pd.DataFrame.from_dict(result_dict, orient = 'index', columns = ['Buy_hold_Value', 'Portfolio_value', 'count_buy', 'count_sell', 'Is_More_Buys'])
df_results['Difference'] = df_results['Portfolio_value'] - df_results['Buy_hold_Value']
df_results['Buy_Hold_%'] = (df_results['Buy_hold_Value'] - 10000)/10000
df_results['Portfolio_%'] = (df_results['Portfolio_value'] - 10000)/10000
df_results['Difference_%'] = df_results['Portfolio_%'] - df_results['Buy_Hold_%']

df_buys = df_results[(df_results['Is_More_Buys'] == True) & (df_results['Difference_%'] > 0)]
df_results.sort_values(by = 'Difference_%', ascending = False)

,Buy_hold_Value,Portfolio_value,count_buy,count_sell,Is_More_Buys,Difference,Buy_Hold_%,Portfolio_%,Difference_%
HAL,15335.327876,49329.406546,17,0,Y,33994.078670,0.533533,3.932941,3.399408
TTD,17791.392119,38233.998667,26,2,Y,20442.606548,0.779139,2.823400,2.044261
EPAM,5038.974144,19349.629803,13,0,Y,14310.655659,-0.496103,0.934963,1.431066
JBHT,11283.518728,22945.397532,15,0,Y,11661.878804,0.128352,1.294540,1.166188
EBAY,17780.028304,29096.136216,10,4,Y,11316.107912,0.778003,1.909614,1.131611
...,...,...,...,...,...,...,...,...,...
HWM,118503.423806,14419.922718,20,0,Y,-104083.501087,10.850342,0.441992,-10.408350
VST,130129.206778,10567.756724,14,1,Y,-119561.450054,12.012921,0.056776,-11.956145
NVDA,157578.414957,23460.154725,59,1,Y,-134118.260232,14.757841,1.346015,-13.411826
PLTR,173270.004272,6506.343703,4,4,N,-166763.660570,16.327000,-0.349366,-16.676366
